<a href="https://colab.research.google.com/github/papago2355/AI-capstone/blob/main/cnn_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt

In [3]:
inputs = pd.read_csv('drive/MyDrive/content/train.csv')
labels = pd.read_csv('drive/MyDrive/content/train_label.csv')
test = pd.read_csv('drive/MyDrive/content/test.csv')

In [4]:
time_series_length = pd.concat([inputs['EMAIL'].value_counts()])
shortest_length = time_series_length[-1]

In [5]:
arranged_labels = []
for id in inputs['EMAIL'].unique():
            idx = inputs['EMAIL'][inputs['EMAIL'] == id].index
            start_idx = idx[0]
            end_idx = idx[-1]
            inputs.drop(list((range(start_idx + shortest_length , end_idx+1))), axis=0, inplace=True)
            inputs = inputs.reset_index(drop=True)
            label_idx = labels['SAMPLE_EMAIL'][labels['SAMPLE_EMAIL'] == id].index[0]
            arranged_labels.append(labels['DIAG_NM'][label_idx])

In [6]:
del_col = [ 'EMAIL', 'summary_date',
                   'activity_class_5min', 'activity_met_1min',
                   'sleep_hr_5min', 'sleep_hypnogram_5min', 'sleep_rmssd_5min', 'timezone', 'sleep_total',
                   'CONVERT(activity_class_5min USING utf8)', 'CONVERT(activity_met_1min USING utf8)',
                   'CONVERT(sleep_hr_5min USING utf8)', 'CONVERT(sleep_hypnogram_5min USING utf8)',
                   'CONVERT(sleep_rmssd_5min USING utf8)']
inputs.drop(del_col, axis=1, inplace=True)
test.drop(del_col, axis=1, inplace=True)

In [7]:
inputs = inputs.drop(columns = ['activity_average_met', 'activity_cal_active', 'activity_cal_total','activity_steps', 'activity_score',
                                    'activity_score_move_every_hour', 'activity_daily_movement','activity_met_min_high',
                                    'activity_score_stay_active', 'activity_total', 'activity_met_min_low','activity_medium',
                                    'activity_score_training_frequency','activity_steps', 'activity_score',
                                    'activity_score_training_volume','sleep_efficiency','sleep_duration','sleep_hr_lowest',
                                    'sleep_midpoint_time','sleep_awake','sleep_deep','sleep_rem','sleep_restless','sleep_temperature_delta',
                                    'sleep_score_total','sleep_score','activity_score_meet_daily_targets','sleep_period_id','activity_inactivity_alerts','activity_score_recovery_time'])
test = test.drop(columns = ['activity_average_met', 'activity_cal_active', 'activity_cal_total','activity_steps', 'activity_score',
                                    'activity_score_move_every_hour', 'activity_daily_movement','activity_met_min_high',
                                    'activity_score_stay_active', 'activity_total', 'activity_met_min_low','activity_medium',
                                    'activity_score_training_frequency','activity_steps', 'activity_score',
                                    'activity_score_training_volume','sleep_efficiency','sleep_duration','sleep_hr_lowest',
                                    'sleep_midpoint_time','sleep_awake','sleep_deep','sleep_rem','sleep_restless','sleep_temperature_delta',
                                    'sleep_score_total','sleep_score','activity_score_meet_daily_targets','sleep_period_id','activity_inactivity_alerts','activity_score_recovery_time'])

In [8]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(inputs)
inputs = scaler.transform(inputs)

In [9]:
arranged_labels = np.reshape(arranged_labels,(-1,1))

In [10]:
import pandas as pd 
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

le = OneHotEncoder()
le.fit(arranged_labels)
arranged_labels = le.fit_transform(arranged_labels).toarray()

In [62]:
le.categories_

[array(['CN', 'Dem', 'MCI'], dtype='<U3')]

In [11]:
# cnn lstm model


x = inputs.reshape((148,35,22))

In [12]:
y = arranged_labels

In [13]:
import os
import pandas as pd
import numpy as np
import tensorflow as tf
import keras
from keras.models import Sequential, Model
from keras.layers import Dense, LSTM, Conv1D, MaxPooling1D, Dropout, Flatten, BatchNormalization, Input, Convolution2D, Activation,TimeDistributed
from keras.layers import Input, multiply, concatenate, Activation, Masking, Reshape
from keras.layers import GlobalAveragePooling1D, Permute, Dropout
import random
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from tqdm import tqdm

In [14]:
def generate_model():
    ip = Input(shape=(35, 22))

    x = Masking()(ip)
    x = LSTM(units = 50, return_sequences = True)(x)
    x = LSTM(units=50)(x)
    x = Dropout(0.5)(x)

  
    y = Reshape((770,1))(ip)
    y = Conv1D (kernel_size=3, filters=128, strides=3, padding='valid',kernel_initializer='he_uniform')(y)
    y = BatchNormalization()(y)
    y = Activation('relu')(y)

    
    y = Conv1D (kernel_size=3, filters=256, padding='same', kernel_initializer='he_uniform')(y)
    y = BatchNormalization()(y)
    y = Activation('relu')(y)
    y = MaxPooling1D(pool_size=3, strides=3)(y)
   
    y = Conv1D (kernel_size=3, filters=128, padding='same', kernel_initializer='he_uniform')(y)
    y = BatchNormalization()(y)
    y = Activation('relu')(y)
    y = MaxPooling1D(pool_size=3, strides=3)(y)
    
     
    
    y = Dropout(0.5)(y)
    y = Flatten()(y)
    
    x = concatenate([x, y])

    out = Dense(3, activation='softmax')(x)

    model = Model(ip, out)
    #model.summary()
    return model
model = generate_model()
model.summary()


Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 35, 22)]     0                                            
__________________________________________________________________________________________________
reshape (Reshape)               (None, 770, 1)       0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1d (Conv1D)                 (None, 256, 128)     512         reshape[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 256, 128)     512         conv1d[0][0]                     
______________________________________________________________________________________________

In [15]:
import numpy as np
from sklearn.model_selection import TimeSeriesSplit

tscv = TimeSeriesSplit(max_train_size=None, n_splits=5)


for train_index, test_index in tscv.split(x):
    print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = x[train_index], x[test_index]
    y_train, y_test = y[train_index], y[test_index]

TRAIN: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27] TEST: [28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51]
TRAIN: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47
 48 49 50 51] TEST: [52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75]
TRAIN: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47
 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71
 72 73 74 75] TEST: [76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99]
TRAIN: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47
 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71
 72 73 74 75 76 77 78 79 80 81 82

In [16]:
from keras.optimizers import SGD
num_models=25
model_list=[]

for i in tqdm(range(num_models)):
    model = generate_model()
    model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
    model.fit(x,y, epochs=15, batch_size=16,validation_data=(X_test,y_test))
    model_list.append(model)
    model.save(f"model_{i}.h5")

  0%|          | 0/25 [00:00<?, ?it/s]

Epoch 1/15
10/10 [==============================] - 29s 299ms/step - loss: 2.3553 - accuracy: 0.4322 - val_loss: 0.9611 - val_accuracy: 0.8333
Epoch 2/15
10/10 [==============================] - 0s 18ms/step - loss: 1.1753 - accuracy: 0.7176 - val_loss: 4.6942 - val_accuracy: 0.1250
Epoch 3/15
10/10 [==============================] - 0s 18ms/step - loss: 0.4741 - accuracy: 0.8173 - val_loss: 0.1874 - val_accuracy: 0.9583
Epoch 4/15
10/10 [==============================] - 0s 18ms/step - loss: 0.1750 - accuracy: 0.9248 - val_loss: 0.5965 - val_accuracy: 0.7083
Epoch 5/15
10/10 [==============================] - 0s 17ms/step - loss: 0.1340 - accuracy: 0.9475 - val_loss: 0.0293 - val_accuracy: 1.0000
Epoch 6/15
10/10 [==============================] - 0s 18ms/step - loss: 0.0906 - accuracy: 0.9824 - val_loss: 0.0360 - val_accuracy: 1.0000
Epoch 7/15
10/10 [==============================] - 0s 18ms/step - loss: 0.0959 - accuracy: 0.9677 - val_loss: 0.0202 - val_accuracy: 1.0000
Epoch 8/15


  4%|▍         | 1/25 [00:33<13:12, 33.01s/it]

Epoch 1/15
10/10 [==============================] - 10s 274ms/step - loss: 2.0902 - accuracy: 0.5356 - val_loss: 1.2260 - val_accuracy: 0.3333
Epoch 2/15
10/10 [==============================] - 0s 19ms/step - loss: 0.9539 - accuracy: 0.6689 - val_loss: 0.5219 - val_accuracy: 0.7500
Epoch 3/15
10/10 [==============================] - 0s 19ms/step - loss: 0.4486 - accuracy: 0.8009 - val_loss: 0.7655 - val_accuracy: 0.6667
Epoch 4/15
10/10 [==============================] - 0s 19ms/step - loss: 0.2184 - accuracy: 0.9162 - val_loss: 1.2462 - val_accuracy: 0.4583
Epoch 5/15
10/10 [==============================] - 0s 18ms/step - loss: 0.2225 - accuracy: 0.9049 - val_loss: 0.1337 - val_accuracy: 0.9167
Epoch 6/15
10/10 [==============================] - 0s 18ms/step - loss: 0.2002 - accuracy: 0.9486 - val_loss: 0.1550 - val_accuracy: 0.9583
Epoch 7/15
10/10 [==============================] - 0s 18ms/step - loss: 0.1241 - accuracy: 0.9720 - val_loss: 1.1835 - val_accuracy: 0.4583
Epoch 8/15


  8%|▊         | 2/25 [00:47<10:28, 27.32s/it]

...

 92%|█████████▏| 23/25 [06:20<00:34, 17.45s/it]

Epoch 1/15
10/10 [==============================] - 12s 495ms/step - loss: 1.9921 - accuracy: 0.5304 - val_loss: 1.8169 - val_accuracy: 0.3750
Epoch 2/15
10/10 [==============================] - 0s 19ms/step - loss: 0.7723 - accuracy: 0.8027 - val_loss: 0.8378 - val_accuracy: 0.8333
Epoch 3/15
10/10 [==============================] - 0s 18ms/step - loss: 0.6104 - accuracy: 0.7489 - val_loss: 4.0703 - val_accuracy: 0.1250
Epoch 4/15
10/10 [==============================] - 0s 17ms/step - loss: 0.2231 - accuracy: 0.9186 - val_loss: 0.2317 - val_accuracy: 0.9167
Epoch 5/15
10/10 [==============================] - 0s 18ms/step - loss: 0.1594 - accuracy: 0.9178 - val_loss: 0.5416 - val_accuracy: 0.7500
Epoch 6/15
10/10 [==============================] - 0s 18ms/step - loss: 0.1407 - accuracy: 0.9363 - val_loss: 0.3275 - val_accuracy: 0.8333
Epoch 7/15
10/10 [==============================] - 0s 17ms/step - loss: 0.0722 - accuracy: 0.9628 - val_loss: 0.0565 - val_accuracy: 1.0000
Epoch 8/15


 96%|█████████▌| 24/25 [06:36<00:16, 16.91s/it]

Epoch 1/15
10/10 [==============================] - 9s 272ms/step - loss: 2.0025 - accuracy: 0.5293 - val_loss: 0.6378 - val_accuracy: 0.7917
Epoch 2/15
10/10 [==============================] - 0s 19ms/step - loss: 0.9918 - accuracy: 0.6871 - val_loss: 1.0192 - val_accuracy: 0.7083
Epoch 3/15
10/10 [==============================] - 0s 19ms/step - loss: 0.4209 - accuracy: 0.8449 - val_loss: 1.1503 - val_accuracy: 0.4583
Epoch 4/15
10/10 [==============================] - 0s 19ms/step - loss: 0.3057 - accuracy: 0.8694 - val_loss: 1.0624 - val_accuracy: 0.5417
Epoch 5/15
10/10 [==============================] - 0s 18ms/step - loss: 0.1604 - accuracy: 0.9457 - val_loss: 0.1809 - val_accuracy: 0.9167
Epoch 6/15
10/10 [==============================] - 0s 18ms/step - loss: 0.0938 - accuracy: 0.9655 - val_loss: 0.1613 - val_accuracy: 0.9167
Epoch 7/15
10/10 [==============================] - 0s 18ms/step - loss: 0.0911 - accuracy: 0.9650 - val_loss: 0.0262 - val_accuracy: 1.0000
Epoch 8/15
1

100%|██████████| 25/25 [06:49<00:00, 16.39s/it]


In [39]:
test_loss, test_acc = model.evaluate(X_test, y_test)

print("Test accuracy", test_acc)
print("Test loss", test_loss)

1/1 [==============================] - 0s 23ms/step - loss: 0.0229 - accuracy: 1.0000
Test accuracy 1.0
Test loss 0.022874727845191956


In [74]:
yhat = model.predict(x)
train_label = []
for i in range(0,148):
    if yhat[i][0] > yhat[i][1] and yhat[i][0] > yhat[i][2]:
        train_label.append('CN')
    elif yhat[i][1] > yhat[i][0] and yhat[i][1] > yhat[i][2]:
        train_label.append('Dem')
    elif yhat[i][2] > yhat[i][1] and yhat[i][2] > yhat[i][0]:
        train_label.append('MCI')

In [76]:
arranged_labels = le.inverse_transform(arranged_labels).ravel()

In [77]:
from sklearn.metrics import f1_score
f1_score(arranged_labels, train_label, average='macro')

1.0

In [40]:
test = pd.read_csv('drive/MyDrive/content/test.csv')

In [41]:
time_series_length = pd.concat([test['EMAIL'].value_counts()])
shortest_length = time_series_length[-1]

In [42]:
shortest_length

35

In [43]:
test.shape

(29092, 65)

In [44]:
for id in test['EMAIL'].unique():
            idx = test['EMAIL'][test['EMAIL'] == id].index
            start_idx = idx[0]
            end_idx = idx[-1]
            test.drop(list((range(start_idx + shortest_length , end_idx+1))), axis=0, inplace=True)
            test = test.reset_index(drop=True)

In [45]:
test.shape

(16485, 65)

In [46]:
del_col = [ 'EMAIL', 'summary_date',
                   'activity_class_5min', 'activity_met_1min',
                   'sleep_hr_5min', 'sleep_hypnogram_5min', 'sleep_rmssd_5min', 'timezone', 'sleep_total',
                   'CONVERT(activity_class_5min USING utf8)', 'CONVERT(activity_met_1min USING utf8)',
                   'CONVERT(sleep_hr_5min USING utf8)', 'CONVERT(sleep_hypnogram_5min USING utf8)',
                   'CONVERT(sleep_rmssd_5min USING utf8)']

test.drop(del_col, axis=1, inplace=True)

In [47]:
test = test.drop(columns = ['activity_average_met', 'activity_cal_active', 'activity_cal_total','activity_steps', 'activity_score',
                                    'activity_score_move_every_hour', 'activity_daily_movement','activity_met_min_high',
                                    'activity_score_stay_active', 'activity_total', 'activity_met_min_low','activity_medium',
                                    'activity_score_training_frequency','activity_steps', 'activity_score',
                                    'activity_score_training_volume','sleep_efficiency','sleep_duration','sleep_hr_lowest',
                                    'sleep_midpoint_time','sleep_awake','sleep_deep','sleep_rem','sleep_restless','sleep_temperature_delta',
                                    'sleep_score_total','sleep_score','activity_score_meet_daily_targets','sleep_period_id','activity_inactivity_alerts','activity_score_recovery_time'])

In [48]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(test)
test = scaler.transform(test)

In [49]:
test.shape

(16485, 22)

In [50]:
x_test = test.reshape((471,35,22))

In [57]:
yhat = model.predict(x_test)

In [58]:
yhat

array([[0.85308594, 0.00952704, 0.13738701],
       [0.848369  , 0.02231917, 0.12931189],
       [0.9777876 , 0.00298412, 0.01922837],
       ...,
       [0.9160201 , 0.00113155, 0.08284844],
       [0.65066195, 0.00536154, 0.34397656],
       [0.95998365, 0.00324477, 0.03677147]], dtype=float32)

In [55]:
test_labels = pd.read_csv('drive/MyDrive/content/sample_submission.csv')

In [63]:
for i in range(0,471):
    if yhat[i][0] > yhat[i][1] and yhat[i][0] > yhat[i][2]:
        test_labels['DIAG_NM'][i] = 'CN'
    elif yhat[i][1] > yhat[i][0] and yhat[i][1] > yhat[i][2]:
        test_labels['DIAG_NM'][i] = 'Dem'
    elif yhat[i][2] > yhat[i][1] and yhat[i][2] > yhat[i][0]:
        test_labels['DIAG_NM'][i] = 'MCI'

In [67]:
yhat[0:10]

array([[8.53085935e-01, 9.52703599e-03, 1.37387007e-01],
       [8.48369002e-01, 2.23191679e-02, 1.29311889e-01],
       [9.77787614e-01, 2.98412330e-03, 1.92283690e-02],
       [8.87867332e-01, 2.88218190e-03, 1.09250516e-01],
       [1.64465740e-01, 9.95118450e-03, 8.25583100e-01],
       [3.02240461e-01, 3.00893880e-04, 6.97458625e-01],
       [9.99244928e-01, 1.15034691e-05, 7.43548328e-04],
       [6.62525892e-01, 8.91426578e-03, 3.28559905e-01],
       [6.54598832e-01, 6.53248327e-03, 3.38868707e-01],
       [9.85482812e-01, 5.44328417e-04, 1.39728673e-02]], dtype=float32)

In [68]:
inv_s2 = le.inverse_transform(yhat).ravel()

In [69]:
inv_s2

array(['CN', 'CN', 'CN', 'CN', 'MCI', 'MCI', 'CN', 'CN', 'CN', 'CN',
       'MCI', 'CN', 'MCI', 'CN', 'CN', 'CN', 'CN', 'CN', 'CN', 'CN', 'CN',
       'CN', 'MCI', 'CN', 'CN', 'MCI', 'CN', 'CN', 'CN', 'CN', 'CN',
       'MCI', 'CN', 'CN', 'CN', 'CN', 'CN', 'MCI', 'CN', 'Dem', 'MCI',
       'CN', 'MCI', 'CN', 'CN', 'CN', 'CN', 'CN', 'MCI', 'MCI', 'CN',
       'MCI', 'MCI', 'CN', 'MCI', 'CN', 'MCI', 'CN', 'MCI', 'CN', 'MCI',
       'CN', 'CN', 'MCI', 'MCI', 'CN', 'CN', 'CN', 'CN', 'CN', 'MCI',
       'CN', 'CN', 'MCI', 'CN', 'CN', 'CN', 'MCI', 'CN', 'CN', 'CN', 'CN',
       'CN', 'CN', 'MCI', 'CN', 'CN', 'MCI', 'CN', 'CN', 'CN', 'CN', 'CN',
       'CN', 'CN', 'CN', 'MCI', 'MCI', 'CN', 'CN', 'CN', 'MCI', 'CN',
       'CN', 'MCI', 'CN', 'CN', 'MCI', 'MCI', 'CN', 'CN', 'CN', 'Dem',
       'CN', 'CN', 'CN', 'MCI', 'MCI', 'MCI', 'MCI', 'MCI', 'CN', 'MCI',
       'CN', 'CN', 'MCI', 'CN', 'CN', 'CN', 'CN', 'Dem', 'MCI', 'CN',
       'MCI', 'CN', 'CN', 'MCI', 'MCI', 'CN', 'MCI', 'CN', 'CN', 'MCI

In [64]:
test_labels.to_csv('drive/MyDrive/content/predict15.csv', index=False)